In [71]:
import sys
sys.setrecursionlimit(15000)

import torch
import torch.nn.functional as F
from torch import nn
import numpy as np
from torch.autograd import Variable
from torchvision.datasets.mnist import MNIST
device='cuda:4'
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
from path import Path
import torch.utils.data as data
from imageio import imread
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F 
import matplotlib.pyplot as plt
%matplotlib inline 
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import *
import os
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [72]:
seed=4
df=pd.read_csv("../../Multi-modality/Model/BCLL/uni model/MLP/preprocessed_BCLL.csv")
no_of_rows,no_of_cols=df.shape
print (no_of_rows,no_of_cols)
df.head()

4655 23


,A,B,C,D,E,F,G,H,I,J,...,N,O,P,Q,R,S,T,U,V,W
0,1009_at,HINT1,1145.5,1014.9,1103.1,850.4,886.6,859.3,1228.6,1231.9,...,763.7,1382.4,1008.2,1025.9,1197.0,735.2,865.3,485.0,811.1,783.0
1,100_g_at,RABGGTA,169.0,200.6,196.3,151.4,167.8,111.3,164.9,240.6,...,277.2,172.7,163.6,194.0,117.1,130.5,164.2,212.2,108.4,114.7
2,1011_s_at,YWHAE,25.9,31.7,43.3,40.9,40.8,26.7,45.1,62.0,...,12.5,53.6,72.8,59.6,33.3,35.8,45.3,68.0,56.7,39.3
3,1012_at,KAT2B,28.0,34.4,13.2,28.3,15.7,56.5,35.2,25.0,...,20.4,11.6,15.9,5.9,18.2,28.8,55.0,35.9,25.2,18.8
4,1013_at,SMAD5,17.2,3.0,6.0,9.9,10.0,16.7,1.5,10.0,...,1.4,6.5,12.3,11.3,9.2,2.1,4.7,1.3,5.4,3.8


In [73]:
header_of_pd=['C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W']
X=np.array(df[header_of_pd])
print(X.shape)
print((df['B'][0].lower()))

(4655, 21)
hint1


In [74]:
class Net(torch.nn.Module):
    def __init__(self, n_feature,out_features):
        super(Net, self).__init__()
        self.hidden1 = torch.nn.Linear(n_feature,900)
        self.hidden2 = torch.nn.Linear(900,600)
        self.hidden3 = torch.nn.Linear(600,400)
        self.hidden4 = torch.nn.Linear(400,100)
        self.hidden5 =torch.nn.Linear (100,10)
        self.predict = torch.nn.Linear(10,out_features)
        self.bn1 = nn.BatchNorm1d(num_features=900)
        self.bn2 = nn.BatchNorm1d(num_features=600)
        self.bn3 = nn.BatchNorm1d(num_features=400)
        self.bn4 = nn.BatchNorm1d(num_features=100)
        self.bn5 = nn.BatchNorm1d(num_features=10)
    def forward(self, x):
        x = F.relu(self.hidden1(x))
#         x=self.bn1(x)
        x = F.relu(self.hidden2(x))
#         x=self.bn2(x)
        x =F.relu(self.hidden3(x))
        #x=self.bn3(x)
        x =F.relu(self.hidden4(x))
      

        return x
MLP_model=Net(21,3)
MLP_model.load_state_dict(torch.load('../../Multi-modality/Model/BCLL/uni model/MLP/1MLP_BCLL.pt'))


In [75]:
MLP_model(torch.rand(4,21)).shape

torch.Size([4, 100])

In [76]:
gene_name_mlp=[]
features_mlp_=[]
for f in range(X.shape[0]):
    print(f)
    gene_name_mlp.append(df['B'][f].lower())
    feature_from_model=MLP_model(torch.tensor(X[f].reshape(1,-1),dtype=torch.float32))
    to_append=feature_from_model[0].data.numpy().tolist()
    features_mlp_.append(to_append)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100


3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761


In [77]:
print(len(features_mlp_))
print(len(gene_name_mlp))

4655
4655


In [78]:
dataframe1 = pd.DataFrame({'gene_name':gene_name_mlp,'feature_MLP':features_mlp_})
tags= dataframe1['feature_MLP'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'tag_' + str(x))
dataframe1=pd.concat([dataframe1['gene_name'], tags[:]], axis=1)
dataframe1.head()
dataframe1.to_csv('BCLL_MLP_features_ankit.csv' ,encoding='utf-8', index=False)

In [79]:
q=pd.read_csv('BCLL_MLP_features_ankit.csv')
q.head()
#MLP FEATURES COMPLETED
len(q)

4655

In [80]:
lstm = nn.LSTM(input_size=4,hidden_size=300,bidirectional=True) #input_size hidden_size num_layers 
lstm2=nn.LSTM(input_size=600,hidden_size=300)
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        
        self.fc1=nn.Linear(300, 500)
        self.bn1=nn.BatchNorm1d(num_features=500)
        
        self.relu=nn.ReLU(inplace=True)
        self.fc2=nn.Linear(500, 200)
        self.bn2=nn.BatchNorm1d(num_features=200)

        self.fc3=nn.Linear(200, 100)
        self.bn3=nn.BatchNorm1d(num_features=100)
        
        self.fc4=nn.Linear(100, 3)
        self.sigm=nn.Sigmoid()
        #self.dropout=nn.Dropout(0.2)
        

    def forward(self, x):
        y=self.fc3(self.relu(self.fc2(self.relu(self.fc1(x)))))
        return y
device='cuda:4'
model1=lstm.to(device)
model2=MLP().to(device)
model3=lstm2.to(device)
model1.load_state_dict(torch.load('../../Multi-modality/Model/BCLL/uni model/BiLSTM/2AnkitBiLSTMBCLL1.pt'))
model2.load_state_dict(torch.load('../../Multi-modality/Model/BCLL/uni model/BiLSTM/2AnkitBiLSTMBCLL2.pt'))
model3.load_state_dict(torch.load('../../Multi-modality/Model/BCLL/uni model/BiLSTM/2AnkitBiLSTMBCLL3.pt'))

In [81]:
def letterToIndex(letter):
    return all_letters.find(letter)
def letterToTensor(letter):
  tensor=torch.zeros(1,4)
  tensor[0][letterToIndex(letter)] = 1
  return tensor
def lineToTensor(line):
  tensor=torch.zeros(len(line),1,4)
  for li,letters in enumerate(line):
    tensor[li][0][letterToIndex(letters)]=1
  return(tensor)

In [82]:
g=pd.read_csv('../../Multi-modality/Model/BCLL/uni model/MLP/preprocessed_BCLL.csv')
mapz=pd.read_csv('../../Multi-modality/Model/BCLL/uni model/BiLSTM/sequence.csv')
key=mapz['gene'].tolist()
sequence=mapz['sequence'].tolist()

dictionary = dict(zip(key, sequence))

In [83]:
all_letters="ACTG"
 

gene_name_lstm=[]
gene_sequence_lstm=[]
gene_lstm_features=[]
for V in (range(len(g['B']))):
    print(V)
    j=g['B'][V]
    GN=j.lower()
    if GN not in dictionary.keys():
        continue
    lin2TenGene=np.array(lineToTensor(dictionary[GN][:70]))
#     print(lin2TenGene.shape) #70 1 4
    gene_name_lstm.append(j.lower())
    gene_sequence_lstm.append(lin2TenGene)
    inp=torch.tensor(lin2TenGene).to(device)

    h0 = torch.zeros(2, 1, 300).to(device)   #(num_layers * num_directions, batch, hidden_size)
    c0 = torch.zeros(2, 1, 300).to(device)
    h_x2=c_x2=torch.zeros(1, 1, 300).to(device)
    output, (hn, cn) = model1(inp, (h0, c0))  
    output,(_,_)=model3(output,(h_x2,c_x2))
    
    q1,q2,q3=output.shape
    out_last=((output[q1-1]))
    After_fc=model2(out_last)
    gene_lstm_features.append(After_fc[0].cpu().data.numpy())
    
            
#             After_fc=model2(out_last)
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073


3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726


In [84]:
# len(gene_lstm_features)
len(gene_name_lstm)

4275

In [85]:
dataframe2 = pd.DataFrame({'gene_name':gene_name_lstm,'feature_LSTM':gene_lstm_features})
# dataframe2.to_csv('ILD_LSTM_features.csv')
tags= dataframe2['feature_LSTM'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'tag_' + str(x))
dataframe2=pd.concat([dataframe2['gene_name'], tags[:]], axis=1)
dataframe2.to_csv('BCLL_LSTM_features_ankit.csv' ,encoding='utf-8', index=False)
dataframe2.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99
0,hint1,-114.235878,42.092331,28.084738,-6.915589,-264.673157,-112.969284,244.293396,-243.060028,177.637634,...,264.639771,-137.051743,239.206879,-286.823090,-249.158676,-108.895470,1.616297,187.885925,-46.757500,-178.767319
1,rabggta,-96.716530,40.485027,14.467666,4.841335,-231.772842,-108.878204,220.188766,-212.098724,151.771713,...,236.870804,-116.660057,214.831741,-255.783127,-222.912292,-98.694870,3.222843,172.100494,-25.469044,-156.375778
2,ywhae,-30.591722,7.940422,23.717236,10.390911,-15.187041,-20.519796,10.701858,-13.258877,52.855568,...,46.496918,-37.264877,45.935513,-26.483770,-6.032434,6.683128,-8.354677,18.291792,2.855942,-20.667261
3,kat2b,-38.674809,10.573558,20.799767,8.663656,-15.912493,-20.277807,9.189360,-13.391596,55.776970,...,51.836746,-44.357887,49.981579,-27.125193,-7.041780,6.086931,-11.960100,16.976311,1.014287,-23.555027
4,smad5,-50.431614,11.698717,41.556259,19.373199,-22.984467,-33.346626,11.359543,-20.751783,81.790749,...,69.264946,-58.623772,68.915756,-43.378422,-7.953577,16.305567,-14.265181,26.523077,7.761844,-32.814716


In [86]:
uv=pd.read_csv("BCLL_LSTM_features_ankit.csv")
print(len(uv))

uv.head()
#LSTM FEATURES COMPLETED

4275


,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99
0,hint1,-114.235880,42.092330,28.084738,-6.915589,-264.673160,-112.969284,244.293400,-243.060030,177.637630,...,264.639770,-137.051740,239.206880,-286.823100,-249.158680,-108.895470,1.616297,187.885930,-46.757500,-178.767320
1,rabggta,-96.716530,40.485027,14.467666,4.841335,-231.772840,-108.878204,220.188770,-212.098720,151.771710,...,236.870800,-116.660060,214.831740,-255.783130,-222.912300,-98.694870,3.222843,172.100500,-25.469044,-156.375780
2,ywhae,-30.591722,7.940422,23.717236,10.390911,-15.187041,-20.519796,10.701858,-13.258877,52.855568,...,46.496918,-37.264877,45.935513,-26.483770,-6.032433,6.683128,-8.354677,18.291792,2.855942,-20.667261
3,kat2b,-38.674810,10.573558,20.799767,8.663656,-15.912493,-20.277807,9.189360,-13.391596,55.776970,...,51.836746,-44.357887,49.981580,-27.125193,-7.041780,6.086931,-11.960100,16.976310,1.014287,-23.555027
4,smad5,-50.431614,11.698717,41.556260,19.373200,-22.984467,-33.346626,11.359543,-20.751783,81.790750,...,69.264946,-58.623770,68.915760,-43.378420,-7.953577,16.305567,-14.265181,26.523077,7.761844,-32.814716


In [61]:
dfs=pd.read_csv('../../Multi-modality/Model/BCLL/uni model/MLP/BCLL_PDB3D_GENETOIDB.csv')
final_df=dfs[pd.notnull(dfs['Cross-reference (PDB)'])]
Gene_names=list(final_df['Gene names  (primary )'])
PDB_idz=list(final_df['Cross-reference (PDB)'])
gene_names=[]
for i in range (len(Gene_names)):
    gene_names.append(Gene_names[i].lower())
dictionary = dict(zip(gene_names, PDB_idz))
inp=pd.read_csv('../../Multi-modality/Model/BCLL/uni model/MLP/preprocessed_BCLL.csv')
BCLL_gene_names=list(inp['B'])
BCLL_gene_names=[i.lower() for i in BCLL_gene_names]
dictionary[BCLL_gene_names[3]]

'1CM0;1JM4;1N72;1WUG;1WUM;1ZS5;2RNW;2RNX;3GG3;4NSQ;5FDZ;5FE0;5FE1;5FE2;5FE3;5FE4;5FE5;5FE6;5FE7;5FE8;5FE9;5LVQ;5LVR;5MKX;'

In [62]:
loc_3D=Path('/home1/pratik/Multi-modality/testrun')

In [63]:
def load_as_float(path):
    f = Image.open(path)
    f=f.resize((100,100))
    rgb = f.convert('RGB')
    return(np.array(rgb))

In [64]:
!nvidia-smi

Mon Jun 17 16:32:12 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0  On |                  N/A |
|  0%   30C    P8    18W / 250W |   1386MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 24%   43C    P2    58W / 250W |    933MiB / 11172MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [65]:
NUM_CLASSES = 3
NUM_ROUTING_ITERATIONS = 3
def softmax(input, dim=1):
    transposed_input = input.transpose(dim, len(input.size()) - 1)
    softmaxed_output = F.softmax(transposed_input.contiguous().view(-1, transposed_input.size(-1)), dim=-1)
    return softmaxed_output.view(*transposed_input.size()).transpose(dim, len(input.size()) - 1)

class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, num_route_nodes, in_channels, out_channels, kernel_size=None, stride=None,
                 num_iterations=NUM_ROUTING_ITERATIONS):
        super(CapsuleLayer, self).__init__()

        self.num_route_nodes = num_route_nodes
        self.num_iterations = num_iterations

        self.num_capsules = num_capsules

        if num_route_nodes != -1:
            self.route_weights = nn.Parameter(torch.randn(num_capsules, num_route_nodes, in_channels, out_channels))
        else:
            self.capsules = nn.ModuleList(
                [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=0) for _ in
                 range(num_capsules)])

    def squash(self, tensor, dim=-1):
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm)
        return scale * tensor / torch.sqrt(squared_norm)

    def forward(self, x):
        if self.num_route_nodes != -1:
#             print("x",x[None, :, :, None, :].shape)
#             print("route",self.route_weights[:, None, :, :, :].shape)
            priors = x[None, :, :, None, :] @ self.route_weights[:, None, :, :, :]
#             print(priors.shape)

            logits = Variable(torch.zeros(*priors.size())).to(device)
#             print("logits-"+str(logits.shape))
            for i in range(self.num_iterations):
                probs = softmax(logits, dim=2)
                outputs = self.squash((probs * priors).sum(dim=2, keepdim=True))
#                 print("onii",outputs.shape)

                if i != self.num_iterations - 1:
                    delta_logits = (priors * outputs).sum(dim=-1, keepdim=True)
#                     print("war",delta_logits.shape)
                    logits = logits + delta_logits
    
        else:
            outputs = [capsule(x).view(x.size(0), -1, 1) for capsule in self.capsules]
#             print("outputs-"+str(outputs[0].shape))
            outputs = torch.cat(outputs, dim=-1)
            outputs = self.squash(outputs)

        return outputs
class CapsuleNet(nn.Module):
    def __init__(self):
        super(CapsuleNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=16, stride=1)
        self.pool=nn.MaxPool2d(3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=12, stride=1)
        
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=9, stride=1)
        self.relu=nn.ReLU()
#         self.dummy_linear=nn.Linear(199712,32*16*16)
        self.primary_capsules = CapsuleLayer(num_capsules=8, num_route_nodes=-1, in_channels=256, out_channels=32,
                                             kernel_size=9, stride=2)
        self.digit_capsules = CapsuleLayer(num_capsules=NUM_CLASSES, num_route_nodes=9248, in_channels=8,
                                           out_channels=16)

        self.decoder = nn.Sequential(
            nn.Linear(16 * NUM_CLASSES, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 784),
            nn.Sigmoid()
        )

    def forward(self, x, y=None):
        x =F.relu(self.conv3((F.relu(self.conv2(self.pool(F.relu(self.conv1(x), inplace=True)))))))
#         print(x.shape)
        x = self.primary_capsules(x)
#         print(x.shape)
        
#         dummy=x.permute(0,2,1)
#         dummy=dummy_linear(dummy)
#         x=dummy.permute(0,2,1)
        x = self.digit_capsules(x).squeeze().transpose(0, 1)



       

        return x

device = 'cuda:4'
model_3d=CapsuleNet().to(device)
model_3d.load_state_dict(torch.load('../../Multi-modality/Model/BCLL/uni model/CapN/1AugmentedBCLL_capsulenet3d_ankit.pt',map_location='cpu'))

In [66]:
u=model_3d(torch.rand(1,3,200,200).to(device))
u.shape

torch.Size([16, 3])

In [67]:
u=model_3d(torch.rand(5,3,200,200).to(device))
u.shape

torch.Size([5, 3, 16])

In [68]:
s=0
gene_name_for3d=[]
feature_for_3d=[]
uz=0
for i in BCLL_gene_names:
    print(uz)
    uz=uz+1
    a=0
    if i  in dictionary.keys():
        pdb_id_=dictionary[i]
        pdbid_list=pdb_id_.split(';')
        for j in pdbid_list: 
            q=j.lower()
            exists = os.path.isdir(loc_3D/q)
            if exists:
                image_loc=loc_3D/q
                images= sorted((image_loc).files('*.png'))
                if(len(images)>=4):
                    imp=np.zeros((200,200,3))            
                    imp[0:100,0:100,:]=np.array(load_as_float(images[0]))
                    imp[0:100,100:200,:]=np.array(load_as_float(images[1]))
                    imp[100:200,0:100,:]=np.array(load_as_float(images[2]))
                    imp[100:200,100:200,:]=np.array(load_as_float(images[3]))
                    
                    input_for_cp=torch.tensor(imp).permute(2,0,1).unsqueeze_(0).float()
                    
                    output_feature=model_3d(input_for_cp.to(device)).reshape(-1).cpu().data.numpy()
                    
                    gene_name_for3d.append(i)
                    feature_for_3d.append(output_feature)
                    
                    s=s+1
                    break
         

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062


3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702


In [69]:
dataframe3 = pd.DataFrame({'gene_name':gene_name_for3d,'feature_3d':feature_for_3d})

tags= dataframe3['feature_3d'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'tag_' + str(x))
dataframe3=pd.concat([dataframe3['gene_name'], tags[:]], axis=1)
dataframe3.to_csv('AnkitBCLL_3d.csv' ,encoding='utf-8', index=False)
dataframe3.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_38,tag_39,tag_40,tag_41,tag_42,tag_43,tag_44,tag_45,tag_46,tag_47
0,hint1,0.000128,0.003666,0.000086,-0.000142,0.013800,0.000378,-0.000040,3.907078e-03,-0.000168,...,0.000020,-0.000014,-0.008416,0.000020,-0.000204,0.005934,-0.000014,0.000322,0.003235,0.000067
1,ywhae,-0.000080,0.000155,0.003896,0.000124,0.001357,0.003743,0.000142,-5.598767e-04,-0.002663,...,0.003278,-0.000108,-0.001078,0.002478,0.000050,0.000687,0.005183,-0.000012,0.000014,-0.001313
2,kat2b,-0.000067,0.003084,0.003566,-0.000042,0.005316,0.004484,-0.000035,8.720114e-04,-0.003647,...,0.002714,-0.000130,-0.002824,0.003957,0.000125,0.003588,0.005358,0.000133,-0.000062,0.000689
3,cib1,0.000136,0.000098,0.024427,0.000034,-0.000053,0.021380,-0.000461,1.194030e-04,-0.022567,...,0.025997,0.000306,0.000063,0.020553,0.000282,-0.000167,0.023225,-0.000222,-0.000087,-0.007981
4,cyp1a1,-0.000320,0.000653,0.009505,-0.000159,0.000983,0.010270,-0.000024,5.357184e-07,-0.007808,...,0.006218,-0.000164,-0.000459,0.007649,-0.000432,0.000244,0.009911,-0.000121,0.000433,-0.000752


In [70]:
xx=pd.read_csv('AnkitBCLL_3d.csv')
xx

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_38,tag_39,tag_40,tag_41,tag_42,tag_43,tag_44,tag_45,tag_46,tag_47
0,hint1,0.000128,0.003666,0.000086,-0.000142,0.013800,0.000378,-0.000040,3.907078e-03,-0.000168,...,0.000020,-1.383306e-05,-0.008416,0.000020,-0.000204,0.005934,-0.000014,0.000322,0.003235,0.000067
1,ywhae,-0.000080,0.000155,0.003896,0.000124,0.001357,0.003743,0.000142,-5.598767e-04,-0.002663,...,0.003278,-1.082131e-04,-0.001078,0.002478,0.000050,0.000687,0.005183,-0.000012,0.000014,-0.001313
2,kat2b,-0.000067,0.003084,0.003566,-0.000042,0.005316,0.004484,-0.000035,8.720113e-04,-0.003647,...,0.002714,-1.301899e-04,-0.002824,0.003957,0.000125,0.003588,0.005358,0.000133,-0.000062,0.000689
3,cib1,0.000136,0.000098,0.024427,0.000034,-0.000053,0.021380,-0.000461,1.194030e-04,-0.022567,...,0.025997,3.055718e-04,0.000063,0.020553,0.000282,-0.000167,0.023225,-0.000222,-0.000087,-0.007981
4,cyp1a1,-0.000320,0.000653,0.009505,-0.000159,0.000983,0.010270,-0.000024,5.357184e-07,-0.007808,...,0.006218,-1.636300e-04,-0.000459,0.007649,-0.000432,0.000244,0.009911,-0.000121,0.000433,-0.000752
5,txk,0.000009,0.001135,0.005009,-0.000324,0.003482,0.004106,0.000210,1.188198e-03,-0.004859,...,0.004560,1.814781e-04,-0.003638,0.003199,0.000135,0.001517,0.005233,0.000089,0.000489,-0.001404
6,top1,-0.000043,-0.000021,0.017037,0.000074,-0.000042,0.011043,0.000072,-5.117618e-06,-0.014493,...,0.019993,1.721387e-04,-0.000019,0.015542,0.000004,0.000012,0.016191,-0.000020,-0.000020,-0.006916
7,srpk1,0.000101,0.004597,0.000730,0.000178,0.007833,0.001740,-0.000217,9.072372e-04,-0.001377,...,0.000703,-3.017822e-04,-0.005768,0.000189,-0.000341,0.003750,0.001521,0.000076,0.000569,-0.001007
8,cxcr2,0.000077,-0.000062,0.014628,-0.000094,0.000022,0.015218,0.000030,1.279913e-05,-0.017318,...,0.015785,-1.014765e-04,0.000067,0.014333,0.000192,-0.000059,0.014677,0.000196,-0.000031,-0.008270
9,timp3,-0.000263,-0.000608,0.025164,0.000196,-0.000116,0.018097,-0.000290,-1.479233e-04,-0.023797,...,0.023328,1.953483e-05,0.000450,0.017850,0.000161,-0.000067,0.020420,-0.000095,0.000052,-0.009543
